<a href="https://colab.research.google.com/github/SalehMehrabani/PyTorch-Deep-Learning/blob/main/NN4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Workflow


In [16]:
import torch 
from torch import nn
import matplotlib.pyplot as plt

In [18]:
device= "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data: Preparing and Loading


In [23]:
#Create Known parameter
weight = 0.7
bias = 0.3

#Create  dataset
start=0
end=1
step= 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

### Splitting Data into Training (Training and Validation set) and Test sets

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)